# SSEEventDispatcher

> Event routing system with namespace support, pattern matching, and middleware pipeline. Enables decoupled event handling with priority-based execution and wildcard routing.

In [ ]:
#| default_exp dispatcher

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import asyncio
from typing import Callable, Dict, Any, List, Optional, Set, Union
from dataclasses import dataclass
import re

In [ ]:
#| export
@dataclass
class SSEEvent:
    """Represents an SSE event with metadata."""
    type: str
    data: Dict[str, Any]
    namespace: Optional[str] = None
    priority: int = 0
    timestamp: Optional[str] = None
    
    @property
    def full_type(self):
        """Get the full event type including namespace."""
        if self.namespace:
            return f"{self.namespace}:{self.type}"
        return self.type

In [ ]:
#| export
class SSEEventDispatcher:
    """
    Decoupled event routing system with namespace support,
    middleware, filtering, and priority-based handling.
    """
    
    def __init__(self):
        """Initialize the SSE Event Dispatcher."""
        self._handlers: Dict[str, List[tuple[int, Callable]]] = {}
        self._middleware: List[Callable] = []
        self._filters: List[Callable] = []
        self._transformers: List[Callable] = []
        self._namespaces: Set[str] = set()
        
    def register_namespace(
        self,
        namespace: str  # Namespace name to register for event organization
    ):
        """Register a namespace for event organization."""
        self._namespaces.add(namespace)
        
    def on(
        self,
        event_pattern: str,  # Event pattern (supports wildcards: *, **)
        priority: int = 0  # Handler priority (higher runs first)
    ):
        """
        Decorator to register an event handler with pattern matching.
        """
        def decorator(
            handler: Callable  # Event handler function to register
        ):
            """Register the handler and return it unchanged."""
            self.add_handler(event_pattern, handler, priority)
            return handler
        return decorator
    
    def add_handler(
        self,
        event_pattern: str,  # Event pattern (e.g., "job:*", "**:completed")
        handler: Callable,  # Handler function
        priority: int = 0  # Handler priority
    ):
        """
        Add an event handler with pattern matching support.
        """
        if event_pattern not in self._handlers:
            self._handlers[event_pattern] = []
        
        self._handlers[event_pattern].append((priority, handler))
        self._handlers[event_pattern].sort(key=lambda x: x[0], reverse=True)
    
    def add_middleware(
        self,
        middleware: Callable  # Function that takes (event, next) and calls next(event)
    ):
        """
        Add middleware that processes events before handlers.
        """
        self._middleware.append(middleware)
    
    def add_filter(
        self,
        filter_func: Callable[[SSEEvent], bool]  # Function that returns True to process event
    ):
        """
        Add a filter to control which events are processed.
        """
        self._filters.append(filter_func)
    
    def add_transformer(
        self,
        transformer: Callable[[SSEEvent], SSEEvent]  # Function that transforms an event
    ):
        """
        Add a transformer to modify events before processing.
        """
        self._transformers.append(transformer)
    
    def _match_pattern(
        self,
        pattern: str,  # Pattern to match against (supports wildcards)
        event_type: str  # Event type string to test
    ) -> bool:  # True if pattern matches event type
        """
        Check if an event type matches a pattern.
        
        Patterns:
        - exact: "job:created" matches only "job:created"
        - wildcard: "job:*" matches "job:created", "job:updated", etc.
        - deep wildcard: "**:created" matches any namespace with "created"
        - full wildcard: "*" matches everything
        """
        if pattern == "*" or pattern == "**":
            return True
            
        # Convert pattern to regex
        regex_pattern = pattern.replace("**", ".*").replace("*", "[^:]*")
        regex_pattern = f"^{regex_pattern}$"
        
        return bool(re.match(regex_pattern, event_type))
    
    async def dispatch(
        self,
        event: Union[SSEEvent, Dict[str, Any]]  # Event to dispatch (SSEEvent or dict)
    ) -> List[Any]:  # List of handler results
        """
        Dispatch an event through the processing pipeline.            
        """
        # Convert dict to SSEEvent if needed
        if isinstance(event, dict):
            event = SSEEvent(
                type=event.get("type", "unknown"),
                data=event.get("data", {}),
                namespace=event.get("namespace"),
                priority=event.get("priority", 0),
                timestamp=event.get("timestamp")
            )
        
        # Apply filters
        for filter_func in self._filters:
            if not filter_func(event):
                return []  # Event filtered out
        
        # Apply transformers
        for transformer in self._transformers:
            event = transformer(event)
        
        # Process through middleware chain
        async def process_event(
            evt: SSEEvent  # Event to process through handlers
        ):
            """Process event through all matching handlers."""
            results = []
            
            # Find matching handlers
            for pattern, handlers in self._handlers.items():
                if self._match_pattern(pattern, evt.full_type):
                    for priority, handler in handlers:
                        if asyncio.iscoroutinefunction(handler):
                            result = await handler(evt)
                        else:
                            result = handler(evt)
                        if result:
                            results.append(result)
            
            return results
        
        # Apply middleware
        current_processor = process_event
        for middleware in reversed(self._middleware):
            next_processor = current_processor
            async def wrapped_processor(
                evt: SSEEvent,  # Event to process
                next_proc: Callable = next_processor  # Next processor in chain
            ):
                """Apply middleware and call next processor."""
                if asyncio.iscoroutinefunction(middleware):
                    return await middleware(evt, next_proc)
                else:
                    return middleware(evt, lambda e: asyncio.create_task(next_proc(e)))
            current_processor = wrapped_processor
        
        return await current_processor(event)
    
    def clear_handlers(
        self,
        pattern: Optional[str] = None  # Specific pattern to clear, or None for all
    ):
        """Clear handlers for a specific pattern or all handlers."""
        if pattern:
            self._handlers.pop(pattern, None)
        else:
            self._handlers.clear()

In [ ]:
event_dispatcher = SSEEventDispatcher()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()